In [0]:
import io
import os
import re
import keras
import tensorflow as tf
import numpy as np # used for handling numbers
import pandas as pd # used for handling the dataset
df= pd.read_csv('drive/My Drive/Colab Notebooks/dataset/labeled_outs.txt',delimiter=",",header=None)
#kw=pd.read_csv('drive/My Drive/Colab Notebooks/dataset/temp_frequent.txt',header=None)
# Dataset is now stored in a Pandas Dataframe

In [0]:
df.columns=['label','text']
df.text=df.text.astype(str)
df.label=df.label.astype(str)
df=df.sample(frac=1)
docs=df['text'].values
labels=df['label'].values

In [0]:

from sklearn.feature_extraction.text import CountVectorizer
input_dim=15000
count_vect = CountVectorizer(max_features = input_dim)
X_train_counts = count_vect.fit_transform(docs)

In [4]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape
X_train_tfidf = X_train_tfidf.toarray()
X_train_tfidf = np.reshape(X_train_tfidf, (X_train_tfidf.shape[0], 1, X_train_tfidf.shape[1]))
print(X_train_tfidf.shape)

(91795, 1, 10000)


In [0]:

kw.columns=['words']
dfreq={}
for k in kw['words']:
  dfreq[k]=0
print(len(dfreq))

19999


In [0]:
for d in docs:
  words=re.split('[\s]',d)
  words=np.unique(words)
  for w in words:
    if w in dfreq:
      dfreq[w]=dfreq[w]+1

tf_idf = []
for d in docs:
  words=re.split('[\s]',d)
  N=len(words)
  tfreq={}
  for w in words:
    if w in tfreq:
      tfreq[w]=tfreq[w]+1
    else:
      tfreq[w]=1
  temp=[]
  for w in dfreq:
    if w in tfreq:
      tf=tfreq[w]*1.0/N
      idf=np.log(len(docs)*1.0/dfreq[w])
      temp.append(tf*idf)
    else:
      temp.append(0.0)
  tf_idf.append(temp)

tf_idf = np.array(tf_idf)
input_dim=len(tf_idf[0])
print(input_dim)
tf_idf = np.reshape(tf_idf, (tf_idf.shape[0], 1, tf_idf.shape[1]))

19999


In [5]:

from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
print(labels)
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(labels)
encoded_Y = encoder.transform(labels)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)
print(dummy_y)


['sports' 'economy' 'economy' ... 'arts_literature' 'arts_literature'
 'state']
[[0. 0. 0. ... 0. 1. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train_tfidf, dummy_y, test_size=0.18)

In [0]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)


(75271, 1, 10000)
(75271, 8)
(16524, 1, 10000)
(16524, 8)


In [0]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

model = Sequential()
model.add(LSTM(2048, input_shape=(1,input_dim),return_sequences=True))
model.add(LSTM(1024,return_sequences=True,dropout=0.3))
model.add(LSTM(512,return_sequences=True,dropout=0.3))
model.add(LSTM(128,return_sequences=True,dropout=0.3))
model.add(LSTM(64,return_sequences=True))
model.add(LSTM(32))
model.add(Dense(8, activation='softmax'))

In [0]:

model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=['acc'])
# summarize the model
print(model.summary())

In [0]:
history = model.fit(X_train, y_train, epochs=15,batch_size=256,shuffle=False, verbose=1,validation_split=0.1)
#model.save_weights('./checkpoints/text_classifier_rnn_tfidf')
!mkdir -p saved_model
model.save('saved_model/text_classifier_rnn_tfidf')

In [0]:
#model.load_weights('./checkpoints/text_classifier_rnn_tfidf')
new_model = tf.keras.models.load_model('saved_model/text_classifier_rnn_tfidf')

In [10]:
results = model.evaluate(X_test, y_test,batch_size=256)
print(results)

16524/16524 [==============================] - 3s 164us/step
[0.49347963264738204, 0.9040789008140564]


In [0]:
tf_idf.shape

(17687, 1, 19999)